<a href="https://colab.research.google.com/github/debalina-chowdhury/Machine-Learning-in-90-days/blob/master/MEPS_HC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
import pandas as pd
import re
import math
nvc = 'No_Value_Code'

In [185]:
def modify_item(line,l1):
  words=''
  if '=' not in line:
    if (line[0]in ['"',"'"] and line[-1] not in ['"', "'"]):
      # print(998)
      words=line[1:]
      print('case 1:')
      print('words initial:', words)
    if (line[0] not in ['"',"'"] and line[-1] in ['"',"'"]):
      # print(999)
      words=line[:-1]
      print('case 2:')
    if (line[0] in ['"',"'"] and line[-1] in ['"',"'"]):
      # print(1000)
      words=line[1:-1]
      print('case 3:')
    print('last l1 element:', l1[-1])
    l1[-1][1]+=words
    print('added:', l1[-1])
  else:
    # print(1000)
    varcode = line.split('=',1)[0].strip()
    words = line.split('=',1)
    # print(varcode)
    if len(words)==1:
      print('words:', words)
      pairs = [[varcode, '']]
    else:
      pairs = [[varcode, words[1].strip()[1:-1]]]
      if words[1]!='':
        if words[1][-1] not in ['"',"'"]:
          pairs = [[varcode, words[1].strip()[1:]]]
    l1.extend(pairs)
    return l1

In [186]:
def variables_MEPSHC_file(sas_file_path):
  # Read the .sas file
  # sas_file_path = 'sample_data/2017_sample_adult_interview_vardesc.txt'
  with open(sas_file_path, 'r') as file:
      sas_code = file.read()
      # print(sas_code)
      eof_index = sum(1 for _ in file)
  with open(sas_file_path, 'r') as file:
    sas_lines = file.readlines()
  l1=[]
  pattern = r'* LABEL STATEMENTS;'


  line_indices = [i + 1 for i, line in enumerate(sas_code.split('\n')) if re.search(re.escape(pattern), line, re.IGNORECASE)]
  print(line_indices)



  for j, line in enumerate(sas_lines, start = line_indices[-1]):
        line = sas_lines[j].strip()
        # print(line)
        if line:
          if ';' in line:
            print('end')
            break
          else:
            if line.startswith('LABEL'):
              line=line.replace('LABEL','')

            modify_item(line, l1)

  l1=[inner_list[-2:] for inner_list in l1]
  df_format = pd.DataFrame(l1, columns=['VarCode', 'VarDesc'])
  return df_format


In [187]:
# sas_file_path = "sample_data/MEPS_HC_2021_h224su.txt"
# # data = pd.read_sas(sas_file_path)
# df1=variables_MEPSHC_file(sas_file_path)
# df1

In [188]:
def MEPSHC_Vardesc(full_consolidated_file_path, med_cond_file_path, jobs_file_path):
  full_year_consolidated_df = variables_MEPSHC_file(full_consolidated_file_path)
  full_year_consolidated_df.insert(0, 'File', 'Full Year Consolidated')
  med_cond_df = variables_MEPSHC_file(med_cond_file_path)
  med_cond_df.insert(0, 'File', 'Medical Conditions')
  jobs_df = variables_MEPSHC_file(jobs_file_path)
  jobs_df.insert(0, 'File', 'Jobs')
  MEPSHC_Vardesc_df = pd.concat([full_year_consolidated_df, med_cond_df, jobs_df], ignore_index = True)

  MEPSHC_Vardesc_df = MEPSHC_Vardesc_df.drop_duplicates()
  MEPSHC_Vardesc_df.insert(0, 'Year', '2021')
  MEPSHC_Vardesc_df.insert(0, 'DatasetShortName', 'MEPS_HC')
  MEPSHC_Vardesc_df.index+=1
  return MEPSHC_Vardesc_df

In [189]:
full_consolidated_file_path = "sample_data/MEPS_HC_2021_h233su.txt"
med_cond_file_path = "sample_data/MEPS_HC_2021_med_cond_h231su.txt"
jobs_file_path = "sample_data/MEPS_HC_2021_jobs_h231su.txt"
MEPSHC_Vardesc_df = MEPSHC_Vardesc(full_consolidated_file_path, med_cond_file_path, jobs_file_path)

MEPSHC_Vardesc_df.head()

[3218]
end
[306]
end
[414]
end


,DatasetShortName,Year,File,VarCode,VarDesc
1,MEPS_HC,2021,Full Year Consolidated,DUID,PANEL # + ENCRYPTED DU IDENTIFIER
2,MEPS_HC,2021,Full Year Consolidated,PID,PERSON NUMBER PORTION OF PERSID
3,MEPS_HC,2021,Full Year Consolidated,DUPERSID,PERSON ID (DUID + PID)
4,MEPS_HC,2021,Full Year Consolidated,PANEL,PANEL NUMBER
5,MEPS_HC,2021,Full Year Consolidated,FAMID31,FAMILY ID (STUDENT MERGED IN) - R3/1


In [190]:
MEPSHC_Vardesc_df.groupby('File').count()

,DatasetShortName,Year,VarCode,VarDesc
File,,,,
Full Year Consolidated,1488,1488,1488,1488
Jobs,86,86,86,86
Medical Conditions,32,32,32,32


In [191]:
def modify_item_varcodedet(line,l1):
  line_split = line.split(' ',1)
  varcode = line_split[0].strip()
  varcodedet = line_split[1].strip()
  varcodedet = varcodedet.replace('.','')
  pairs = [[varcode, varcodedet]]
  l1.extend(pairs)
  return l1

In [192]:
def add_varcodedet(sas_file_path):
  with open(sas_file_path, 'r') as file:
      sas_code = file.read()
      # print(sas_code)
      eof_index = sum(1 for _ in file)
  with open(sas_file_path, 'r') as file:
    sas_lines = file.readlines()
  l1=[]
  pattern = r'* FORMAT STATEMENTS;'

  line_indices = [i + 1 for i, line in enumerate(sas_code.split('\n')) if re.search(re.escape(pattern), line, re.IGNORECASE)]
  print(line_indices)
  for j, line in enumerate(sas_lines, start = line_indices[-1]):
        line = sas_lines[j].strip()
        # print(line)
        if line:
          if ';' in line:
            print('end')
            break
          else:
            if line.startswith('FORMAT '):
              line=line.replace('FORMAT ','')

            modify_item_varcodedet(line, l1)

  l1=[inner_list[-2:] for inner_list in l1]
  df_format = pd.DataFrame(l1, columns=['VarCode', 'VarCodeDet'])
  df_format.loc[df_format['VarCode']=='ICD10CDX', 'VarCodeDet'] = '$ICD10CDB'
  df_format.loc[df_format['VarCode'].isin(['CCSR1X', 'CCSR2X', 'CCSR3X']), 'VarCodeDet'] = '$CCSR1B'
  return df_format

In [193]:
# full_consolidated_file_path = "sample_data/MEPS_HC_2021_h224su.txt"
# fcf_varcodedet = add_varcodedet(full_consolidated_file_path)
# fcf_varcodedet.loc[fcf_varcodedet['VarCode']=='PID']
# print(fcf_varcodedet['VarCode'].head())

In [194]:
MEPSHC_Vardesc_df.shape

(1606, 5)

In [195]:
def complete_MEPSHC_Vardesc(full_consolidated_file_path, med_cond_file_path, jobs_file_path, vardesc):
  full_year_consolidated_varcodedet = add_varcodedet(full_consolidated_file_path)
  full_year_consolidated_varcodedet.insert(0, 'File', 'Full Year Consolidated')
  med_cond_varcodedet = add_varcodedet(med_cond_file_path)
  med_cond_varcodedet.insert(0, 'File', 'Medical Conditions')
  jobs_varcodedet = add_varcodedet(jobs_file_path)
  jobs_varcodedet.insert(0, 'File', 'Jobs')
  df_format = pd.concat([full_year_consolidated_varcodedet, med_cond_varcodedet, jobs_varcodedet], ignore_index=False)
  df_format = df_format.drop_duplicates()
  df_merged = pd.merge(vardesc, df_format, on=['VarCode','File'], how='left')
  df_merged['FileVarDetails']=df_merged['File']
  df_merged.reset_index(drop=True, inplace=True)
  df_merged.index+=1
  return df_merged

In [196]:
full_consolidated_file_path = "sample_data/MEPS_HC_2021_h233su.txt"
med_cond_file_path = "sample_data/MEPS_HC_2021_med_cond_h231su.txt"
jobs_file_path = "sample_data/MEPS_HC_2021_jobs_h231su.txt"
MEPSHC_Vardesc_df = complete_MEPSHC_Vardesc(full_consolidated_file_path, med_cond_file_path, jobs_file_path, MEPSHC_Vardesc_df)

MEPSHC_Vardesc_df.head()

[1726]
end
[270]
end
[324]
end


,DatasetShortName,Year,File,VarCode,VarDesc,VarCodeDet,FileVarDetails
1,MEPS_HC,2021,Full Year Consolidated,DUID,PANEL # + ENCRYPTED DU IDENTIFIER,DUIDF,Full Year Consolidated
2,MEPS_HC,2021,Full Year Consolidated,PID,PERSON NUMBER PORTION OF PERSID,PIDF,Full Year Consolidated
3,MEPS_HC,2021,Full Year Consolidated,DUPERSID,PERSON ID (DUID + PID),$DUPERSIDF,Full Year Consolidated
4,MEPS_HC,2021,Full Year Consolidated,PANEL,PANEL NUMBER,PANELF,Full Year Consolidated
5,MEPS_HC,2021,Full Year Consolidated,FAMID31,FAMILY ID (STUDENT MERGED IN) - R3/1,$FAMID31F,Full Year Consolidated


In [197]:
MEPSHC_Vardesc_df.shape

(1606, 7)

In [227]:
def modify_item_values(varcodedet, line,l1):
  words=''
  pairs=[[]]
  # if '/*' not in line or '*/' in line:
  #   pass
  if '=' not in line:
    pass
  else:
    # print(1000)
    line=line.strip(';')
    line=line.strip()
    words = line.split('=',1)
    val = words[0].strip()
    valdesc = words[1].strip()
    if 'CASHP21XF' in varcodedet:
      print(val, valdesc)

    if 'VALID ID' in valdesc:
      val = nvc
    varcodedet=varcodedet.strip()
    if (val[0]=="'" and val[-1]=="'") or (val[0]=='"' and val[-1]=='"'):
      val = val[1:-1]
    if val in [" ",' ']:
      val=nvc
    val = val.replace("' - '", " - ")

    if valdesc!='':
      if valdesc[-1] not in ["'",'"','”']:
        pairs = [[varcodedet, val, valdesc.strip()[1:]]]
    if (valdesc[0]=="'" and valdesc[-1]=="'") or (valdesc[0]=='"' and valdesc[-1]=='"'):
      valdesc = valdesc.strip()[1:-1]

    varcodedet_special = 'BUSNP21XF'
    if re.search('-?\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?\s*-\s*-?\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?', valdesc):
      if varcodedet == varcodedet_special:
        print('case 1:', varcodedet, pairs)
      pairs = [[varcodedet, nvc, "Non categorical_Actual values reported"]]
      # if varcodedet=='CHLDP21XF':
      #   print(varcodedet, pairs)
      # if varcodedet=='FAMINC21F':
      #   print(varcodedet, pairs)

    elif re.search('-?\$\d{1,3}(,\d{3})*(\.\d{2})?', valdesc):
      if varcodedet == varcodedet_special:
        print('case 2:', varcodedet, pairs)
      print('case 2: ', varcodedet, val, valdesc)
      pairs = [[varcodedet, nvc, "Non categorical_Actual values reported"]]
      # print('case 2: ', varcodedet)
    elif re.search(r'\b\d{4} - \d{4}\b', val):
      if varcodedet == varcodedet_special:
        print('case 3:', varcodedet, pairs)
      # print('case 1 varcodedet', varcodedet)
      pairs = [[varcodedet, nvc, valdesc]]
    elif re.search('-?\d+\s*-\s*-?\d+', val):
      if varcodedet == varcodedet_special:
        print('case 4:', varcodedet, pairs)
      # print('case 2 varcodedet', varcodedet)
      if re.search('-?\d+\s*-\s*-?\d+', valdesc):
        if varcodedet == varcodedet_special:
          print('case 5:', varcodedet, pairs)
        valdesc = "Non categorical_Actual values reported"
      pairs = [[varcodedet, nvc, valdesc]]
    elif valdesc=='0.00':
      if varcodedet == varcodedet_special:
        print('case 6:', varcodedet, pairs)
      print('last anomaly: ', varcodedet, val, valdesc)
      valdesc = '0'
      pairs = [[varcodedet, val, valdesc]]
      l1.extend([[varcodedet, nvc, "Non categorical_Actual values reported"]])
    else:
      if varcodedet == varcodedet_special:
        print('case 7:', varcodedet, pairs)
      pairs = [[varcodedet, val, valdesc.strip()]]

    # if varcodedet == 'HRWG31HF':
    #     print(varcodedet, pairs)

    l1.extend(pairs)

    return l1

In [228]:
def variables_details_MEPSHC_file(sas_file_path):
  # Read the .sas file
  # sas_file_path = 'sample_data/2017_sample_adult_interview_sourcevarcat.txt'

  with open(sas_file_path, 'r') as file:
      sas_code_all = file.read()
      eof_index = sum(1 for _ in file)

  pattern3 = r'* VALUE STATEMENTS;'

  line_index = [i + 1 for i, line in enumerate(sas_code_all.split('\n')) if re.search(re.escape(pattern3), line, re.IGNORECASE)]
  print(line_index)
  pattern = r'^VALUE'
  pattern2=r'^     Value'
  # Find all matches
  # matches = re.finditer(pattern, sas_code)
  sas_code = '\n'.join(sas_code_all.split('\n')[line_index[0]:])
  matches = re.finditer(pattern, sas_code, flags=re.MULTILINE)
  # print(matches[0])
  # Get line index of each match
  line_indices = [sas_code.count('\n', 0, match.start()) for match in matches]

  if len(line_indices)==0:
    matches = re.finditer(pattern2, sas_code, flags=re.MULTILINE)
    line_indices = [sas_code.count('\n', 0, match.start()) for match in matches]
  # print('line indices:', line_indices )
  result = [(line_indices[i], line_indices[i+1]) for i in range(len(line_indices)-1)]
  l1=[]
  with open(sas_file_path, 'r') as file:
    sas_lines = file.readlines()[line_index[0]:]
  for index in result:
    f = sas_lines[index[0]].strip().split(' ',1)
    if ' ' in f[-1]:
      f = sas_lines[index[0]].strip().split(' ',2)

    varcodedet=f[1]
    # print(sas_lines[index[0]])
    for i in range(index[0], index[1]):
      line = sas_lines[i].strip()
      if line in ['',';']:
        pass
      else:
        if i == index[0]:
          if len(f)==3:
            # print(f)
            line=f[-1]
            # if varcodedet=='SESPLANYR_LAB':
            #   print(line)
        modify_item_values(varcodedet, line, l1)
  f=sas_lines[line_indices[-1]].strip().split(' ',1)
  if ' ' in f[-1]:
    f=sas_lines[line_indices[-1]].strip().split(' ',2)
  varcodedet=f[1]

  # print(f,varcodedet)
  # print('last varcode:', varcodedet)
  # print(line_indices[-1])
  for j, line_new in enumerate(sas_lines, start = line_indices[-1]):
    # print('in')
    sas_lines[j]=sas_lines[j].strip()
    # print(sas_lines[j])
    if sas_lines[j] in [';']:
      break
    elif sas_lines[j] in ['']:
        pass
    else:
      # print(sas_lines[j])
      if j==line_indices[-1]:
        if len(f)==3:
          line=f[-1]
          # if varcodedet=='SESPLANYR_LAB':
          #     print(line)
          modify_item_values(varcodedet, line, l1)
      else:
        modify_item_values(varcodedet, sas_lines[j], l1)
  df_format = pd.DataFrame(l1, columns=['VarCodeDet', 'ValueCode', 'ValueDesc'])
  df_format=df_format.drop_duplicates()
  df_format = df_format[~df_format['VarCodeDet'].isin(['$CCSRF','$ICD10CD','$CCSR2B','$CCSR3B'])]
  return df_format

In [229]:
sas_file_path = "sample_data/MEPS_HC_2021_h233su.txt"
# data = pd.read_sas(sas_file_path)
df1=variables_details_MEPSHC_file(sas_file_path)
df1.head()

[4710]
case 1: BUSNP21XF [[]]
case 7: BUSNP21XF [[]]
case 1: BUSNP21XF [[]]
case 1: BUSNP21XF [[]]
case 1: BUSNP21XF [[]]
case 1: BUSNP21XF [[]]
0 "0"
312 - 2731 "$312 - $2,731"
2732 - 6839 "$2,732 - $6,839"
6840 - 13021 "$6,840 - $13,021"
13022 - 140857 "$13,022 - $140,857"
last anomaly:  DVTWCP21F 0 0.00
last anomaly:  HHAOFD21F 0 0.00
last anomaly:  HHNOFD21F 0 0.00
last anomaly:  HHNSTL21F 0 0.00
last anomaly:  HHNWCP21F 0 0.00
case 2:  HRWG31HF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG31XF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG42HF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG42XF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG53HF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG53XF -10 -10 HOURLY WAGE >= $105.77
last anomaly:  IPDOFD21F 0 0.00
case 2:  NHRWG31F -10 -10 HOURLY WAGE >= $105.77
case 2:  NHRWG42F -10 -10 HOURLY WAGE >= $105.77
case 2:  NHRWG53F -10 -10 HOURLY WAGE >= $105.77
case 2:  OPDSTL21F 12 $12
case 2:  OPSOFD21F 121 $121
case 2:  OPSSTL21F 12 $12
last anomal

,VarCodeDet,ValueCode,ValueDesc
0,ACCELI42F,-1,-1 INAPPLICABLE
1,ACCELI42F,1,1 YES
2,ACTDTY31F,-15,-15 CANNOT BE COMPUTED
3,ACTDTY31F,-8,-8 DK
4,ACTDTY31F,-7,-7 REFUSED


In [201]:
def assemble_Variable_details(full_consolidated_file_path, med_cond_file_path, jobs_file_path):
  data = [['Non_categorical', nvc, 'Non categorical_Actual values reported']]
  new_df=pd.DataFrame(data, columns=['VarCodeDet','ValueCode','ValueDesc'])
  full_consolidated_var_det = variables_details_MEPSHC_file(full_consolidated_file_path)
  full_consolidated_var_det=pd.concat([full_consolidated_var_det, new_df], ignore_index=False)
  # screener_var_det = add_actual_value_rows(screener_var_det)
  print(full_consolidated_var_det.shape)
  full_consolidated_var_det.insert(0,'File','Full Year Consolidated')
  full_consolidated_var_det['FileVarDetails']='Full Year Consolidated'
  print(full_consolidated_var_det.head())

  med_cond_var_det = variables_details_MEPSHC_file(med_cond_file_path)
  med_cond_var_det=pd.concat([med_cond_var_det, new_df], ignore_index=False)
  print(med_cond_var_det.shape)
  med_cond_var_det.insert(0,'File','Medical Conditions')
  med_cond_var_det['FileVarDetails']='Medical Conditions'

  jobs_file_var_det = variables_details_MEPSHC_file(jobs_file_path)
  jobs_file_var_det=pd.concat([jobs_file_var_det, new_df], ignore_index=False)
  jobs_file_var_det.insert(0,'File','Jobs')
  jobs_file_var_det['FileVarDetails']='Jobs'

  df_var_det = pd.concat([full_consolidated_var_det, med_cond_var_det, jobs_file_var_det], ignore_index = False)
  df_var_det.insert(0, 'Year', '2021')
  df_var_det.insert(0, 'DatasetShortName', 'MEPS_HC')
  df_var_det.reset_index(drop=True, inplace=True)
  df_var_det.index+=1
  return df_var_det

In [202]:
full_consolidated_file_path = "sample_data/MEPS_HC_2021_h233su.txt"
med_cond_file_path = "sample_data/MEPS_HC_2021_med_cond_h231su.txt"
jobs_file_path = "sample_data/MEPS_HC_2021_jobs_h231su.txt"
MEPSHC_Variable_details = assemble_Variable_details(full_consolidated_file_path, med_cond_file_path, jobs_file_path)

MEPSHC_Variable_details.tail()

[4710]
0 "0"
312 - 2731 "$312 - $2,731"
2732 - 6839 "$2,732 - $6,839"
6840 - 13021 "$6,840 - $13,021"
13022 - 140857 "$13,022 - $140,857"
case 2:  HRWG31HF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG31XF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG42HF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG42XF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG53HF -10 -10 HOURLY WAGE >= $105.77
case 2:  HRWG53XF -10 -10 HOURLY WAGE >= $105.77
case 2:  NHRWG31F -10 -10 HOURLY WAGE >= $105.77
case 2:  NHRWG42F -10 -10 HOURLY WAGE >= $105.77
case 2:  NHRWG53F -10 -10 HOURLY WAGE >= $105.77
case 2:  OPDSTL21F 12 $12
case 2:  OPSOFD21F 121 $121
case 2:  OPSSTL21F 12 $12
(6752, 3)
                     File VarCodeDet ValueCode               ValueDesc  \
0  Full Year Consolidated  ACCELI42F        -1         -1 INAPPLICABLE   
1  Full Year Consolidated  ACCELI42F         1                   1 YES   
2  Full Year Consolidated  ACTDTY31F       -15  -15 CANNOT BE COMPUTED   
3  Full Year Consolidated  ACTDTY31

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
7848,MEPS_HC,2021,Jobs,YNOBUSN_M,1,1 BUSINESS SOLD OR CLOSED,Jobs
7849,MEPS_HC,2021,Jobs,YNOBUSN_M,2,2 RETIRED,Jobs
7850,MEPS_HC,2021,Jobs,YNOBUSN_M,3,3 ILLNESS OR INJURY,Jobs
7851,MEPS_HC,2021,Jobs,YNOBUSN_M,91,91 OTHER,Jobs
7852,MEPS_HC,2021,Jobs,Non_categorical,No_Value_Code,Non categorical_Actual values reported,Jobs


In [203]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='FCSZ1231F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
2004,MEPS_HC,2021,Full Year Consolidated,FCSZ1231F,-1,-1 INAPPLICABLE,Full Year Consolidated
2005,MEPS_HC,2021,Full Year Consolidated,FCSZ1231F,1,1 PERSON,Full Year Consolidated
2006,MEPS_HC,2021,Full Year Consolidated,FCSZ1231F,2,2 PERSONS,Full Year Consolidated
2007,MEPS_HC,2021,Full Year Consolidated,FCSZ1231F,3,3 PERSONS,Full Year Consolidated
2008,MEPS_HC,2021,Full Year Consolidated,FCSZ1231F,No_Value_Code,4+ NUMBER OF PERSONS,Full Year Consolidated


In [204]:
import re

# Define the regex pattern
pattern = r'-?\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?\s*-\s*-?\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?'

# Example text containing the pattern
text = "Here are some ranges: $3,316 - $6,034, $500- $1,500, $1,000.50--$2,500.75, $300-$400, and some invalid ones: $1000-$20."

# Use re.findall to find all matches
matches = re.findall(pattern, text)
print(matches)


['$3,316 - $6,034', '$500- $1,500', '$1,000.50--$2,500.75', '$300-$400']


In [205]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='$ICD10CD']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails


In [206]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='$ICD10CDB']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
7027,MEPS_HC,2021,Medical Conditions,$ICD10CDB,-15,-15 CANNOT BE COMPUTED,Medical Conditions
7028,MEPS_HC,2021,Medical Conditions,$ICD10CDB,A08,A08 VIRAL AND OTHER SPECIFIED INTESTINAL INFEC...,Medical Conditions
7029,MEPS_HC,2021,Medical Conditions,$ICD10CDB,A09,"A09 INFECTIOUS GASTROENTERITIS AND COLITIS, UN...",Medical Conditions
7030,MEPS_HC,2021,Medical Conditions,$ICD10CDB,A49,A49 BACTERIAL INFECTION OF UNSPECIFIED SITE,Medical Conditions
7031,MEPS_HC,2021,Medical Conditions,$ICD10CDB,B00,B00 HERPESVIRAL [HERPES SIMPLEX] INFECTIONS,Medical Conditions
...,...,...,...,...,...,...,...
7313,MEPS_HC,2021,Medical Conditions,$ICD10CDB,Z90,"Z90 ACQUIRED ABSENCE OF ORGANS, NOT ELSEWHERE ...",Medical Conditions
7314,MEPS_HC,2021,Medical Conditions,$ICD10CDB,Z91,"Z91 PERSONAL RISK FACTORS, NOT ELSEWHERE CLASS...",Medical Conditions
7315,MEPS_HC,2021,Medical Conditions,$ICD10CDB,Z95,Z95 PRESENCE OF CARDIAC AND VASCULAR IMPLANTS ...,Medical Conditions
7316,MEPS_HC,2021,Medical Conditions,$ICD10CDB,Z96,Z96 PRESENCE OF OTHER FUNCTIONAL IMPLANTS,Medical Conditions


In [207]:
MEPSHC_Vardesc_df.loc[MEPSHC_Vardesc_df['VarCodeDet']=='Non_categorical']

,DatasetShortName,Year,File,VarCode,VarDesc,VarCodeDet,FileVarDetails


In [208]:
!pip install xlsxwriter

In [209]:
def write_files(NSCH_file_loc, Variables_df, data, name):
  df=pd.DataFrame.from_dict(data, orient='index')
  df.columns = [''] * len(df.columns)
  with pd.ExcelWriter(NSCH_file_loc,engine='xlsxwriter') as writer:
      print('NAME:',name)
      Variables_df.to_excel(writer, sheet_name=name)
      df.to_excel(writer, sheet_name='Readme')

In [210]:
data = {'Dataset/Year': 'MEPS_HC 2021', 'source': 'MEPS_HC SAS files', 'url': 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=1%2CHousehold+Full+Year+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Consolidated+Data'}
MEPSHC_variables_loc = 'sample_data/MEPS_HC_2021_Variables_A_sas_DC_2024_05_22_v2.xlsx'
MEPSHC_variable_details_loc = 'sample_data/MEPS_HC_2021_Variable_details_A_sas_DC_2024_05_28.xlsx'
write_files(MEPSHC_variables_loc, MEPSHC_Vardesc_df, data, 'Variables')
write_files(MEPSHC_variable_details_loc, MEPSHC_Variable_details, data, 'Variable details')

NAME: Variables
NAME: Variable details


In [211]:
MEPSHC_Vardesc_df.head()

,DatasetShortName,Year,File,VarCode,VarDesc,VarCodeDet,FileVarDetails
1,MEPS_HC,2021,Full Year Consolidated,DUID,PANEL # + ENCRYPTED DU IDENTIFIER,DUIDF,Full Year Consolidated
2,MEPS_HC,2021,Full Year Consolidated,PID,PERSON NUMBER PORTION OF PERSID,PIDF,Full Year Consolidated
3,MEPS_HC,2021,Full Year Consolidated,DUPERSID,PERSON ID (DUID + PID),$DUPERSIDF,Full Year Consolidated
4,MEPS_HC,2021,Full Year Consolidated,PANEL,PANEL NUMBER,PANELF,Full Year Consolidated
5,MEPS_HC,2021,Full Year Consolidated,FAMID31,FAMILY ID (STUDENT MERGED IN) - R3/1,$FAMID31F,Full Year Consolidated


In [212]:
set(MEPSHC_Vardesc_df['VarCodeDet'].unique()) - set(MEPSHC_Variable_details['VarCodeDet'].unique())

set()

In [213]:
MEPSHC_Variable_details.head()

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
1,MEPS_HC,2021,Full Year Consolidated,ACCELI42F,-1,-1 INAPPLICABLE,Full Year Consolidated
2,MEPS_HC,2021,Full Year Consolidated,ACCELI42F,1,1 YES,Full Year Consolidated
3,MEPS_HC,2021,Full Year Consolidated,ACTDTY31F,-15,-15 CANNOT BE COMPUTED,Full Year Consolidated
4,MEPS_HC,2021,Full Year Consolidated,ACTDTY31F,-8,-8 DK,Full Year Consolidated
5,MEPS_HC,2021,Full Year Consolidated,ACTDTY31F,-7,-7 REFUSED,Full Year Consolidated


In [214]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='IPDOFD21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
3090,MEPS_HC,2021,Full Year Consolidated,IPDOFD21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated
3091,MEPS_HC,2021,Full Year Consolidated,IPDOFD21F,0,0,Full Year Consolidated


In [215]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='DVTWCP21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
1628,MEPS_HC,2021,Full Year Consolidated,DVTWCP21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated
1629,MEPS_HC,2021,Full Year Consolidated,DVTWCP21F,0,0,Full Year Consolidated


In [216]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='HHAOFD21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
2345,MEPS_HC,2021,Full Year Consolidated,HHAOFD21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated
2346,MEPS_HC,2021,Full Year Consolidated,HHAOFD21F,0,0,Full Year Consolidated


In [217]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='HHNOFD21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
2379,MEPS_HC,2021,Full Year Consolidated,HHNOFD21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated
2380,MEPS_HC,2021,Full Year Consolidated,HHNOFD21F,0,0,Full Year Consolidated


In [218]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='HHNSTL21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
2391,MEPS_HC,2021,Full Year Consolidated,HHNSTL21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated
2392,MEPS_HC,2021,Full Year Consolidated,HHNSTL21F,0,0,Full Year Consolidated


In [219]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='HHNWCP21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
2399,MEPS_HC,2021,Full Year Consolidated,HHNWCP21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated
2400,MEPS_HC,2021,Full Year Consolidated,HHNWCP21F,0,0,Full Year Consolidated


In [220]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='VISWCP21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
6615,MEPS_HC,2021,Full Year Consolidated,VISWCP21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated
6616,MEPS_HC,2021,Full Year Consolidated,VISWCP21F,0,0,Full Year Consolidated


In [221]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='OPDSTL21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
4077,MEPS_HC,2021,Full Year Consolidated,OPDSTL21F,0,0,Full Year Consolidated
4078,MEPS_HC,2021,Full Year Consolidated,OPDSTL21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated


In [222]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='OPSOFD21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
4121,MEPS_HC,2021,Full Year Consolidated,OPSOFD21F,0,0,Full Year Consolidated
4122,MEPS_HC,2021,Full Year Consolidated,OPSOFD21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated


In [223]:
MEPSHC_Variable_details.loc[MEPSHC_Variable_details['VarCodeDet']=='OPSSTL21F']

,DatasetShortName,Year,File,VarCodeDet,ValueCode,ValueDesc,FileVarDetails
4133,MEPS_HC,2021,Full Year Consolidated,OPSSTL21F,0,0,Full Year Consolidated
4134,MEPS_HC,2021,Full Year Consolidated,OPSSTL21F,No_Value_Code,Non categorical_Actual values reported,Full Year Consolidated
